In [8]:
import autograd.numpy as np
import autograd.numpy.random as npr
npr.seed(0)

import pandas as pd
import cst

import ssm
from ssm.util import find_permutation
from ssm.plots import gradient_cmap, white_to_color_cmap

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")
sns.set_context("talk")

color_names = [
    "windows blue",
    "red",
    "amber",
    "faded green",
    "dusty purple",
    "orange"
    ]

colors = sns.xkcd_palette(color_names)
cmap = gradient_cmap(colors)


# Speficy whether or not to save figures
save_figures = False

In [11]:
filename = '/home/raeed/data/project-data/smile/cst-gainlag/library/Ford_20180618_COCST_TD.mat'
td = cst.get_cst_dataframe(filename)

In [13]:
display(td)

,monkey,date_time,task,trial_id,result,bin_size,lambda,ct_location,ot_location,idx_startTime,idx_endTime,idx_ctHoldTime,idx_goCueTime,idx_otHoldTime,idx_rewardTime,idx_failTime,cursor_pos,hand_pos,M1_unit_guide,M1_spikes
0,Ford,2018/06/18 08:32:55,CO,11.0,R,0.001,nan,"[0.0, 470.0, 100.0]","[0.0, 415.0, 100.0]",2.0,3232.0,1101.0,2601.0,3033.0,3083.0,nan,"[[43.99231413097475, 609.4093214344089], [43.9...","[[43.99231413097475, 609.4093214344089, -124.6...","[[1.0, 1.0], [1.0, 2.0], [2.0, 1.0], [2.0, 2.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,Ford,2018/06/18 08:32:59,CO,12.0,R,0.001,nan,"[0.0, 470.0, 100.0]","[0.0, 525.0, 100.0]",2.0,3716.0,404.0,2904.0,3417.0,3467.0,nan,"[[2.1983134302915146, 444.62827547945227], [2....","[[2.1983134302915146, 444.62827547945227, -25....","[[1.0, 1.0], [1.0, 2.0], [2.0, 1.0], [2.0, 2.0...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,Ford,2018/06/18 08:33:03,CO,13.0,R,0.001,nan,"[0.0, 470.0, 100.0]","[-55.0, 470.0, 100.0]",2.0,3322.0,342.0,2342.0,3073.0,3123.0,nan,"[[1.760355125332124, 499.2432002691462], [1.74...","[[1.760355125332124, 499.2432002691462, -52.57...","[[1.0, 1.0], [1.0, 2.0], [2.0, 1.0], [2.0, 2.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,Ford,2018/06/18 08:33:06,CO,14.0,R,0.001,nan,"[0.0, 470.0, 100.0]","[55.0, 470.0, 100.0]","[2.0, 547.0]",3204.0,"[59.0, 1081.0]",2581.0,3005.0,3055.0,nan,"[[-10.829086182139132, 474.4217475296349], [-1...","[[-10.829086182139132, 474.4217475296349, -18....","[[1.0, 1.0], [1.0, 2.0], [2.0, 1.0], [2.0, 2.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,Ford,2018/06/18 08:33:09,CO,15.0,R,0.001,nan,"[0.0, 470.0, 100.0]","[0.0, 525.0, 100.0]",2.0,2484.0,307.0,1807.0,2285.0,2335.0,nan,"[[26.149404841682895, 479.7500783166482], [26....","[[26.149404841682895, 479.7500783166482, -26.5...","[[1.0, 1.0], [1.0, 2.0], [2.0, 1.0], [2.0, 2.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,Ford,2018/06/18 09:15:56,CST,396.0,R,0.001,3.5,"[0.0, 470.0, 100.0]",nan,2.0,8371.0,1447.0,1947.0,nan,7947.0,nan,"[[-10.284881147021887, 540.2307586683726], [-1...","[[-10.284881147021887, 540.2307586683726, -79....","[[1.0, 1.0], [1.0, 2.0], [2.0, 1.0], [2.0, 2.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
386,Ford,2018/06/18 09:16:05,CST,397.0,R,0.001,3.5,"[0.0, 470.0, 100.0]",nan,2.0,8143.0,1219.0,1719.0,nan,7719.0,nan,"[[7.602116942656789, 555.2575389232022], [7.59...","[[7.602116942656789, 555.2575389232022, -90.73...","[[1.0, 1.0], [1.0, 2.0], [2.0, 1.0], [2.0, 2.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
387,Ford,2018/06/18 09:16:13,CST,398.0,R,0.001,3.5,"[0.0, 470.0, 100.0]",nan,2.0,7671.0,747.0,1247.0,nan,7247.0,nan,"[[0.7339577487947785, 551.1931615821018], [0.7...","[[0.7339577487947785, 551.1931615821018, -95.1...","[[1.0, 1.0], [1.0, 2.0], [2.0, 1.0], [2.0, 2.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
388,Ford,2018/06/18 09:16:22,CST,399.0,R,0.001,3.5,"[0.0, 470.0, 100.0]",nan,2.0,8548.0,1624.0,2124.0,nan,8124.0,nan,"[[5.198662394194859, 562.9744582358583], [5.21...","[[5.198662394194859, 562.9744582358583, -102.6...","[[1.0, 1.0], [1.0, 2.0], [2.0, 1.0], [2.0, 2.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
